## Computing Isochrones

The following script utilizes the `get_isochrone` function in order to obtain the distance that can be walked in 15 minutes from al lparks, schools and supermarkets in Vienna. These locations were chosen as they represent essential education, leisure, shop locations.

In [1]:
import requests
import pandas as pd
import numpy as np
import os
import sys
import pickle
import matplotlib.pyplot as plt
import folium
import time

sys.path.append('../src/data')
from download_pois_networks import download_POIs
from download_pois_networks import download_street_network, networks_to_gdf, analyze_street_network

In [3]:
def get_isochrone(
    coordinates,                 # Expects either a list [lon, lat] or a string "lon,lat"
    profile="walking",
    contours_minutes=None,
    contours_meters=None,
    contours_colors=None,
    polygons=True,
    access_token="YOUR_ACCESS_TOKEN"
):
    """
    Fetches isochrones from the Mapbox API.

    Args:
        coordinates (list or str): [lon, lat] list or "lon,lat" string.
        profile (str): Travel mode ("driving", "walking", "cycling").
        contours_minutes (list): List of contour times in minutes.
        contours_meters (list): Optional list of contour distances in meters.
        contours_colors (list): Optional hex color codes for each contour.
        polygons (bool): Whether to return polygons or lines.
        access_token (str): Your Mapbox API token.

    Returns:
        dict: API response as JSON.

    Limits:
        - Max 300 requests per minute
        - Max 4 contours per request
        - Max 60 minutes per contour
        - Max 100000 meters per contour
    """
    # Accept both list or pre-formatted string input
    if isinstance(coordinates, list) or isinstance(coordinates, tuple):
        lon, lat = coordinates
        # Validate ranges
        if not (-180 <= lon <= 180 and -90 <= lat <= 90):
            raise ValueError(f"Invalid coordinates: lon={lon}, lat={lat}")
        coord_str = f"{lon},{lat}"
    elif isinstance(coordinates, str):
        coord_str = coordinates
    else:
        raise TypeError("Coordinates must be a [lon, lat] list or 'lon,lat' string")

    # Validate contours limits
    if contours_minutes:
        if len(contours_minutes) > 4:
            raise ValueError("Mapbox allows a maximum of 4 isochrone contours per request.")
        if any(minute > 60 for minute in contours_minutes):
            raise ValueError("Each isochrone contour must be 60 minutes or less.")
    if contours_meters:
        if len(contours_meters) > 4:
            raise ValueError("Mapbox allows a maximum of 4 isochrone contours per request.")
        if any(meter > 100000 for meter in contours_meters):
            raise ValueError("Each isochrone contour must be 100,000 meters or less.")

    # Build the URL
    base_url = "https://api.mapbox.com/isochrone/v1/mapbox/"
    url = f"{base_url}{profile}/{coord_str}?"

    if contours_minutes:
        url += f"contours_minutes={','.join(map(str, contours_minutes))}&"
    elif contours_meters:
        url += f"contours_meters={','.join(map(str, contours_meters))}&"

    if contours_colors:
        url += f"contours_colors={','.join(contours_colors)}&"

    if polygons:
        url += "polygons=true&"

    url += f"access_token={access_token}"
    time.sleep(0.21)  # 0.21 sec per request keeps it under 300/minute safely

    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None


In [5]:
EQUAL_AREA_PROJ = '+proj=cea'
LON_LAT_PROJ = 'EPSG:4326'
MERCATOR_PROJ = 'epsg:3395'

In [7]:
raw_data_path = '../data/raw/'
os.listdir(raw_data_path)

['ViennaAustria_r8_hex_grid.pkl',
 'ViennaAustria_networks.pkl',
 'ViennaAustria.pkl',
 'ViennaAustria_r8_nearest_loc.pkl',
 'ViennaAustria_r8_travel_times.pkl']

In [9]:
with open(f'{raw_data_path}ViennaAustria.pkl','rb') as f:
    gdf_pois = pickle.load(f)

print(gdf_pois.shape)
display(gdf_pois.head(5))

(4015, 373)


addr:city addr:country addr:housenumber addr:postcode  \
node 15079903      Wien           AT              180          1140   
     15080180      Wien           AT             130B          1140   
     43749523      Wien           AT            65-67          1130   
     61274250      Wien           AT               30          1020   
     61274400       NaN          NaN              NaN           NaN   

                         addr:street  atm       brand brand:wikidata  \
node 15079903    Hütteldorfer Straße  yes    Eurospar      Q12309283   
     15080180    Hütteldorfer Straße  NaN  Billa Plus     Q115431077   
     43749523          Amalienstraße  NaN  Billa Plus     Q115431077   
     61274250  Untere Augartenstraße  NaN       Billa        Q537781   
     61274400                    NaN  NaN         NaN            NaN   

                      brand:wikipedia        name  ... food:lokma  \
node 15079903             da:Eurospar    Eurospar  ...        NaN   
     15080180                     NaN  Billa Plus  ...        NaN   
     43749523                     NaN  Billa Plus  ...        NaN   
     61274250  en:Billa (supermarket)       Billa  ...        NaN   
     61274400                     NaN      Nakwon  ...        NaN   

              payment:qr_code service:electricity buffet smokefree winery  \
node 15079903             NaN                 NaN    NaN       NaN    NaN   
     15080180             NaN                 NaN    NaN       NaN    NaN   
     43749523             NaN                 NaN    NaN       NaN    NaN   
     61274250             NaN                 NaN    NaN       NaN    NaN   
     61274400             NaN                 NaN    NaN       NaN    NaN   

              operator:wikipedia building:part seasonal phone_1  
node 15079903                NaN           NaN      NaN     NaN  
     15080180                NaN           NaN      NaN     NaN  
     43749523                NaN           NaN      NaN     NaN  
     61274250                NaN           NaN      NaN     NaN  
     61274400                NaN           NaN      NaN     NaN  

[5 rows x 373 columns]

In [11]:
# Define the tags
tags = {
    'shop': ['supermarket'],
    'leisure': ['park'],
    'amenity': ['school']
}

boundary, pois = download_POIs(place_name="Vienna, Austria", tags=tags)

Total POIs: 4015
Found 1046 shop-supermarket
Found 1110 leisure-park
Found 616 amenity-school


In [14]:
# Filter the previously obtained data for these tags
filtered_pois_by_tag = {}
for tag_key, tag_values in tags.items():
    for tag_value in tag_values:
        tag_name = f"{tag_key}_{tag_value}"
        # Filter POIs that match this tag
        mask = pois[tag_key] == tag_value
        if mask.any():
            filtered_pois_by_tag[tag_name] = pois[mask]

print(len(filtered_pois_by_tag))

3


In [16]:
# Obtain the lat and lon coordinates
processed_pois = {}

for tag, gdf in filtered_pois_by_tag.items():
    if not gdf.empty:
        gdf = gdf.copy() 
        
        gdf_proj = gdf.to_crs(EQUAL_AREA_PROJ)

        # Compute centroid
        gdf_proj['centroid'] = gdf_proj.geometry.centroid

        gdf_centroid_wgs = gdf_proj.set_geometry('centroid').to_crs(LON_LAT_PROJ)

        lon_lat_df = pd.DataFrame({
            'lon': gdf_centroid_wgs.geometry.x,
            'lat': gdf_centroid_wgs.geometry.y
        }, index=gdf.index)

        gdf = pd.concat([gdf, lon_lat_df], axis=1).copy()

        processed_pois[tag] = gdf[['name', 'lon', 'lat']]

In [18]:
# Get isochrones
contours_minutes = [15]  # 15 minutes
contours_colors = ["6706ce"] 
access_token = "pk.eyJ1IjoibWFyaWlha2FybmF1a2giLCJhIjoiY204YWFwMHp3MTkwNjJqc2Z0Nmh5ajhlNiJ9.UyMp2t8mfXEDI2iaf07qeQ"

In [22]:
# Apply the get_isochrone function to all the coordinates
isochrone_results = {}

for tag, df in processed_pois.items():
    print(f"Processing {tag} - {len(df)} POIs")
    tag_results = []

    for idx, row in df.iterrows():
        # Format coordinates as "lon,lat" string for the API
        coord_str = f"{row['lon']},{row['lat']}"

        try:
            result = get_isochrone(
                coordinates=coord_str,
                profile="walking",
                contours_minutes=contours_minutes,
                contours_colors=contours_colors,
                access_token=access_token
            )
            tag_results.append({
                'name': row['name'],
                'lon': row['lon'],
                'lat': row['lat'],
                'isochrone': result
            })
        except Exception as e:
            print(f"Failed for {row['name']}: {e}")

    isochrone_results[tag] = tag_results

Processing shop_supermarket - 1046 POIs
Processing leisure_park - 1110 POIs
Processing amenity_school - 616 POIs


In [65]:
isochrone_results

{'shop_supermarket': [{'name': 'Eurospar',
   'lon': 16.2876675,
   'lat': 48.19696659639973,
   'isochrone': {'features': [{'properties': {'fill-opacity': 0.33,
       'fillColor': '#6706ce',
       'opacity': 0.33,
       'fill': '#6706ce',
       'fillOpacity': 0.33,
       'color': '#6706ce',
       'contour': 15,
       'metric': 'time'},
      'geometry': {'coordinates': [[[16.286668, 48.20602],
         [16.282668, 48.204666],
         [16.279804, 48.20283],
         [16.278668, 48.203062],
         [16.276668, 48.20221],
         [16.275494, 48.20214],
         [16.274035, 48.200599],
         [16.271664, 48.199967],
         [16.270275, 48.198967],
         [16.271855, 48.197967],
         [16.274313, 48.192612],
         [16.277356, 48.190967],
         [16.279667, 48.188483],
         [16.28056, 48.189074],
         [16.281042, 48.188967],
         [16.282241, 48.187967],
         [16.282552, 48.186967],
         [16.284668, 48.185765],
         [16.286668, 48.186742],
     

In [38]:
processed_pois['leisure_park']

name        lon        lat
node     1526007218        Alma Mahler-Werfel-Park  16.395090  48.196644
         6404205450               Rothschildgarten  16.358674  48.251870
         7592682496                    Sophienpark  16.340122  48.197874
         8294326798  Biodiversitätsoase Mandlgasse  16.334467  48.179844
         8494322746                            NaN  16.308866  48.234636
...                                            ...        ...        ...
relation 16705557                              NaN  16.459970  48.223345
         17206316                              NaN  16.346867  48.219496
         17262822               Arne-Karlsson-Park  16.353636  48.221193
         17583687          Mizzi-Langer-Kauba-Park  16.338359  48.201685
         18034554                         Saarpark  16.348770  48.245618

[1110 rows x 3 columns]

In [40]:
processed_pois['amenity_school']

name  \
node     61274271                                    HAK/HAS Augarten   
         66933665                         Volksschule Jochbergengasse   
         76653086                            Museum des Blindenwesens   
         111110224                             Volksschule Marktgasse   
         248527643                                 VS Kindermanngasse   
...                                                               ...   
relation 8353144                             Volksschule Irenäusgasse   
         11533757                                   BHAK/BHAS Wien 10   
         11757829   Hertha Firnberg Schulen für Wirtschaft und Tou...   
         13188134                                GTVS Längenfeldgasse   
         13413284                                  Löwenschule Aspern   

                          lon        lat  
node     61274271   16.372216  48.220502  
         66933665   16.402072  48.279923  
         76653086   16.399272  48.206369  
         111110224  16.357504  48.227375  
         248527643  16.331419  48.217885  
...                       ...        ...  
relation 8353144    16.389030  48.290312  
         11533757   16.364947  48.176846  
         11757829   16.441491  48.241760  
         13188134   16.338999  48.182030  
         13413284   16.479631  48.217484  

[616 rows x 3 columns]

In [27]:
processed_pois['shop_supermarket']

name        lon        lat
node     15079903      Eurospar  16.287668  48.196967
         15080180    Billa Plus  16.298907  48.197763
         43749523    Billa Plus  16.279954  48.191102
         61274250         Billa  16.373589  48.221177
         61274400        Nakwon  16.385663  48.216799
...                         ...        ...        ...
way      1284762848  Billa Plus  16.376779  48.186126
         1284762849  Billa Plus  16.376964  48.185963
         1284892484       Billa  16.376613  48.186024
         1360331543       Penny  16.451360  48.258518
relation 2574831          Hofer  16.371366  48.237117

[1046 rows x 3 columns]

In [ ]:
# Initialize the map (centered somewhere, e.g., Vienna center)
m = folium.Map(location=[48.2082, 16.3738], zoom_start=12)

# Loop through all isochrone results and plot each polygon
for tag, poi_list in isochrone_results.items():
    for poi in poi_list:
        isochrone_geojson = poi['isochrone']
        
        if isochrone_geojson and "features" in isochrone_geojson:
            # Add the isochrone polygon(s) to the map
            folium.GeoJson(
                isochrone_geojson,
                name=f"Isochrone - {poi['name']}",
                style_function=lambda x: {
                    'fillColor': '#6706ce',
                    'color': '#6706ce',
                    'weight': 1,
                    'fillOpacity': 0.3
                }
            ).add_to(m)
            
            # Optional: Add a marker for the POI itself
            folium.Marker(
                location=[poi['lat'], poi['lon']],
                popup=poi['name'],
                icon=folium.Icon(color='blue', icon='info-sign')
            ).add_to(m)

# Add layer control to toggle isochrones
folium.LayerControl().add_to(m)

# Save or display the map
m.save("isochrone_map.html")
m


In [61]:
# Example coordinates for Los Angeles
coordinates = "-118.22258,33.99038"

# Example parameters for 15-minute polygon
contours_minutes = [15]  # Only 15 minutes
contours_colors = ["6706ce"]  # Optional: Single color for the 15-minute contour
access_token = "pk.eyJ1IjoibWFyaWlha2FybmF1a2giLCJhIjoiY204YWFwMHp3MTkwNjJqc2Z0Nmh5ajhlNiJ9.UyMp2t8mfXEDI2iaf07qeQ"

# Call the function
result = get_isochrone(
    coordinates=coordinates,
    profile="driving",  # Change to "walking" or "cycling" if needed
    contours_minutes=contours_minutes,
    contours_colors=contours_colors,
    access_token=access_token
)

# Extract the polygon from the response
if result and "features" in result:
    polygon = result["features"][0]["geometry"]  # Get the geometry of the first feature
    print("Polygon for 15-minute reachable area:")
    print(polygon)
else:
    print("Failed to retrieve the polygon.")

Polygon for 15-minute reachable area:
{'coordinates': [[[-118.21658, 34.043051], [-118.21758, 34.042835], [-118.217983, 34.041783], [-118.219829, 34.040629], [-118.220002, 34.037802], [-118.22058, 34.037426], [-118.221438, 34.037522], [-118.22158, 34.038198], [-118.221816, 34.037616], [-118.223669, 34.037469], [-118.226033, 34.034833], [-118.228995, 34.034965], [-118.22945, 34.03651], [-118.230332, 34.03738], [-118.23058, 34.038911], [-118.23158, 34.036308], [-118.23358, 34.034514], [-118.234474, 34.03438], [-118.235149, 34.031949], [-118.237179, 34.03138], [-118.23758, 34.03058], [-118.23858, 34.031799], [-118.24058, 34.029843], [-118.24258, 34.031636], [-118.24358, 34.032234], [-118.244269, 34.032069], [-118.244384, 34.03138], [-118.243785, 34.03038], [-118.244339, 34.028139], [-118.24558, 34.027736], [-118.24758, 34.028285], [-118.248189, 34.027989], [-118.248783, 34.026583], [-118.250392, 34.026192], [-118.25158, 34.026662], [-118.25258, 34.026049], [-118.253835, 34.02638], [-118.2

In [3]:
import folium

# Create a map centered at the starting coordinates
map_center = [33.99038, -118.22258]
m = folium.Map(location=map_center, zoom_start=13)

# Add the polygon to the map
if result and "features" in result:
    polygon = result["features"][0]["geometry"]
    folium.GeoJson(polygon).add_to(m)

# Save or display the map
m.save("isochrone_map.html")  # Save to an HTML file
m  # Display in a Jupyter notebook